# Creating a map

- png file (2000x2000 size)
- yaml file (same as the other examples, just change the image file name)
- on create_map.py change "map_name" to the name of the png
- run create_map.py
- generates .wbt file to load on webots and .csv file

# TO-DO:

- make presets for circle, square, rectangle, triangle, pentagon, hexagon, hallway, curved hallway, S hallway, V corner, U corner
- make the new map generator
- get all the algorithms
- make all the testing maps

# __________________________________________________

In [8]:
import yaml
from PIL import Image
import numpy as np
import csv

if __name__ == '__main__':
    custom_maps_filepath: str = '../worlds/custom_maps/'
    map_name: str = 'mapW'                             # INPUT

    # Parse the YAML file
    yaml_filepath: str = custom_maps_filepath + map_name + '_config.yaml'
    with open(yaml_filepath, 'r') as stream:
        yaml_data = yaml.safe_load(stream)

    # Process the image to get the coords of the wall pixels
    image_filename: str = yaml_data['image']
    resolution: float = 0.001
    origin: [float, float, float] = yaml_data['origin']
    occupied_thresh: float = 0.65
    max_pixel_value_for_wall: int = int(255*occupied_thresh)
    wall_pixels_coords: [(int, int)] = []

    img: Image = Image.open(custom_maps_filepath + image_filename).convert('L')

    np_img: np.array = np.array(img)
    print(np_img)
    height: int = len(np_img)
    width: int = len(np_img[0])
    for row in range(len(np_img)):
        for col in range(len(np_img[row])):
            if np_img[row][col] <= max_pixel_value_for_wall:
                wall_pixels_coords.append((origin[0] + resolution*col, origin[1] + resolution*(height - row)))

    print('num walls = ', len(wall_pixels_coords))

    # Create and save the coords file
    f = open(custom_maps_filepath + map_name + '_points.csv', 'w', newline='')
    writer = csv.writer(f)
    #    Add the borders
    for x in range(width):
        writer.writerow((origin[0] + resolution*x, origin[1]))
        writer.writerow((origin[0] + resolution*x, origin[1] + resolution*(height - 1)))
    for y in range(1, width-1):
        writer.writerow((origin[0], origin[1] + resolution*y))
        writer.writerow((origin[0] + resolution*(width - 1), origin[1] + resolution*y))
    #    Add the walls
    for coord in wall_pixels_coords:
        writer.writerow(coord)
    f.close()

    # Create and save the new Webots file
    base_map_webots_filepath: str = custom_maps_filepath + 'base_map.wbt'
    f = open(base_map_webots_filepath, 'r')
    webots_str: str = f.read()
    f.close()

    map_webots_filepath: str = custom_maps_filepath + map_name + '.wbt'
    f = open(map_webots_filepath, 'w')
    f.write(webots_str)

    #    Add the rectangular arena
    f.write('RectangleArena {')
    f.write('  translation ' + str(origin[0] + resolution*width/2) + ' ' + str(origin[1] + resolution*height/2) + ' 0.0')
    f.write('  floorSize ' + str(resolution*width) + ' ' + str(resolution*height))
    f.write('  floorTileSize 0.25 0.25')
    f.write('  floorAppearance Parquetry {')
    f.write('    type "light strip"')
    f.write('  }')
    f.write('  wallHeight 0.05')
    f.write('}')

    #    Add the walls
    index: int = 0
    for coord in wall_pixels_coords:
        f.write('Solid {')
        f.write('    translation ' + str(coord[0]) + ' ' + str(coord[1]) + ' 0.025')
        f.write('    children [')
        f.write('        Shape {')
        f.write('            geometry Box {')
        f.write('                size ' + str(resolution) + ' ' + str(resolution) + ' 0.05')
        f.write('            }')
        f.write('        }')
        f.write('    ]')
        f.write('    name "solid' + str(index) + '"')
        f.write('}')
        """
        Solid {
            translation -0.54 -0.06 0.025
            children [
                Shape {
                    geometry Box {
                        size 0.125 0.125 0.05
                    }
                }
            ]
            name "solid1"
        }
        """
        index += 1
    f.close()

ModuleNotFoundError: No module named 'yaml'

# Map generation Algorithm

In [ ]:
public class Room {
    protected int left, right, top, bottom;

    protected int GetWidth() {
        return right - left + 1;
    }

    protected int GetHeight() {
        return top - bottom + 1;
    }

    public Room(int left, int right, int top, int bottom) {
        this.left = left;
        this.right = right;
        this.top = top;
        this.bottom = bottom;
    }

    public int GetLeft() { return left; }
    public int GetRight() { return right; }
    public int GetTop() { return top; }
    public int GetBottom() { return bottom; }

}


public virtual GameObject Draw() {
    GameObject roomContainer = new GameObject("Room");
    Color debugColor = Random.ColorHSV();

    for (int x = left; x <= right; x++) {
        for (int y = bottom; y <= top; y++) {

            GameObject tile = new GameObject("Tile");
            tile.transform.position = new Vector3(x, y, 0);
            tile.transform.localScale = Vector3.one * 6.25f;
            tile.transform.SetParent(roomContainer.transform, true);

            SpriteRenderer sr = tile.AddComponent<SpriteRenderer>();
            sr.sprite = Resources.Load<Sprite>("square");
            sr.color = debugColor;
        }
    }

    return roomContainer;
}


public class BinaryRoom : Room {

    private static int minWidth = 8;
    private static int maxWidth = 20;
    private static int minHeight = 8;
    private static int maxHeight = 20;

    private static int trimTiles = 1;
    private static int corridorMargin = 1;
    private static int minCorridorThickness = 2;

    private bool horizontalSplit;
    private bool verticalSplit;

    private BinaryRoom leftRoom;
    private BinaryRoom rightRoom;

    public BinaryRoom (int left, int right, int top, int bottom) : base(left, right, top, bottom) {
        horizontalSplit = false;
        verticalSplit = false;

        leftRoom = null;
        rightRoom = null;

        if (GetHeight() < minHeight || GetWidth() < minWidth)
            Debug.LogError("Small room");
    }

    public bool IsLeaf() {
        return (horizontalSplit == false) && (verticalSplit == false);
    }
}


public void Split() {
  // Attempt random split
  float rand = Random.value;
  if (rand < 0.5f && GetWidth() >= 2 * minWidth) {
    VerticalSplit();
    return;
  }
  else if (GetHeight() >= 2 * minHeight) {
    HorizontalSplit();
    return;
  } 

  // Force split if theres too much space.
  if (GetWidth() > maxWidth) {
    VerticalSplit();
    return;
  }

  if (GetHeight() > maxHeight) {
    HorizontalSplit();
    return;
  }
}


public override GameObject Draw() {
  if (IsLeaf()) {
    return base.Draw();
  }
  else {
    leftRoom.Draw();
    rightRoom.Draw();
    return null;
  }
}


public override GameObject Draw() {
  if (IsLeaf()) {
    return base.Draw();
  }
  else {
    leftRoom.Draw();
    rightRoom.Draw();
    return null;
  }
}

In [3]:
import math
import numpy as np
import matplotlib.pyplot as plt
import random
import numpy as np
from scipy.ndimage import rotate, zoom
from PIL import Image

from skimage.draw import (
    polygon_perimeter,
    circle_perimeter,
    ellipse_perimeter
)

fig = plt.figure(figsize=(20, 20))
map_border = 100

<Figure size 1440x1440 with 0 Axes>

In [7]:
def make_square():
    map_shape = (2000, 2000)
    map_border = 10
    
    map = np.ones(map_shape, dtype=np.double)
    size = random.randint(120, 400)
    angle = random.randint(0, 360)
    square_matrix = np.ones((size + size // 2, size + size // 2), dtype=np.double)
    
    rr, cc = polygon_perimeter([0, 0, size, size], [0, size, size, 0], shape=square_matrix.shape)
    square_matrix[rr, cc] = 0

    rotated = rotate(square_matrix, angle, reshape=True, mode='constant', cval=1.0, order=1)
    x = random.randint(0 + map_border, map.shape[0] - rotated.shape[0] - map_border)
    y = random.randint(0 + map_border, map.shape[1] - rotated.shape[1] - map_border)
    position = (x + (size // 2), y + (size // 2))
    #map[x:x + rotated.shape[0], y:y + rotated.shape[1]] = rotated

    print("Side length:", size, "px Angle:", angle, "º Center:", position)
    used = [x, y, x + rotated.shape[0], y + rotated.shape[1]]
    return map, used

In [9]:
map, coords = make_square()
map[(map < 0.8)] = 0  # Force non-zero values to be zero

im = Image.fromarray((map * 255).astype(np.uint8))
im.show()
im.save('generated_map.png')

Side length: 263 px Angle: 193 º Center: (598, 1327)


In [4]:
import csv

In [20]:
def create_map(map_name,custom_maps_filepath):
    # Process the image to get the coords of the wall pixels
    image_filename: str = map_name + '_map.png'
    resolution: float = 0.001
    origin: [float, float, float] = [-0.1,-0.1,0]
    occupied_thresh: float = 0.65
    max_pixel_value_for_wall: int = int(255*occupied_thresh)
    wall_pixels_coords: [(int, int)] = []

    img: Image = Image.open(custom_maps_filepath + image_filename).convert('L')

    np_img: np.array = np.array(img)
    height: int = len(np_img)
    width: int = len(np_img[0])
    for row in range(len(np_img)):
        for col in range(len(np_img[row])):
            if np_img[row][col] <= max_pixel_value_for_wall:
                wall_pixels_coords.append((origin[0] + resolution*col, origin[1] + resolution*(height - row)))

    display_name = map_name+'_map'
    print('map = ', display_name)
    print('num walls = ', len(wall_pixels_coords))
    print()

    # Create and save the coords file
    f = open(custom_maps_filepath + map_name + '_points.csv', 'w', newline='')
    writer = csv.writer(f)
    #    Add the borders
    for x in range(width):
        writer.writerow((origin[0] + resolution*x, origin[1]))
        writer.writerow((origin[0] + resolution*x, origin[1] + resolution*(height - 1)))
    for y in range(1, width-1):
        writer.writerow((origin[0], origin[1] + resolution*y))
        writer.writerow((origin[0] + resolution*(width - 1), origin[1] + resolution*y))
    #    Add the walls
    for coord in wall_pixels_coords:
        writer.writerow(coord)
    f.close()

    # Create and save the new Webots file
    base_map_webots_filepath: str = custom_maps_filepath + 'base_map.wbt'
    f = open(base_map_webots_filepath, 'r')
    webots_str: str = f.read()
    f.close()

    map_webots_filepath: str = custom_maps_filepath + map_name + '.wbt'
    f = open(map_webots_filepath, 'w')
    f.write(webots_str)

    #    Add the rectangular arena
    f.write('RectangleArena {')
    f.write('  translation ' + str(origin[0] + resolution*width/2) + ' ' + str(origin[1] + resolution*height/2) + ' 0.0')
    f.write('  floorSize ' + str(resolution*width) + ' ' + str(resolution*height))
    f.write('  floorTileSize 0.25 0.25')
    f.write('  floorAppearance Parquetry {')
    f.write('    type "light strip"')
    f.write('  }')
    f.write('  wallHeight 0.05')
    f.write('}')

    #    Add the walls
    index: int = 0
    for coord in wall_pixels_coords:
        f.write('Solid {')
        f.write('    translation ' + str(coord[0]) + ' ' + str(coord[1]) + ' 0.025')
        f.write('    children [')
        f.write('        Shape {')
        f.write('            geometry Box {')
        f.write('                size ' + str(resolution) + ' ' + str(resolution) + ' 0.05')
        f.write('            }')
        f.write('        }')
        f.write('    ]')
        f.write('    name "solid' + str(index) + '"')
        # f.write('    boundingObject Box {')
        # f.write('        size ' + str(resolution) + ' ' + str(resolution) + ' 0.05')
        # f.write('    }')
        f.write('}')
        """
        Solid {
            translation -0.54 -0.06 0.025
            children [
                Shape {
                    geometry Box {
                        size 0.125 0.125 0.05
                    }
                }
            ]
            name "solid1"
            boundingObject Box {
                size 0.125 0.125 0.05
            }
        }
        """
        index += 1
    f.close()

In [21]:
create_map("Squares",'')

map =  Squares_map
num walls =  11608



In [16]:
maps = ["Squares","Circles","SCorridor","LineCorridor","UCorner","VCorner","Triangles","Mixed"]
filepath: str = ''

for map_name in maps:
    create_map(map_name,filepath)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\Paulo Alexandre\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py", line 3369, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Paulo Alexandre\AppData\Local\Temp\ipykernel_20216\786946796.py", line 5, in <cell line: 4>
    create_map(map_name,filepath)
  File "C:\Users\Paulo Alexandre\AppData\Local\Temp\ipykernel_20216\4202713951.py", line -1, in create_map
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Paulo Alexandre\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py", line 1982, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "C:\Users\Paulo Alexandre\AppData\Roaming\Python\Python310\site-packages\IPython\core\ultratb.py", line 1118, in structured_traceback
    return FormattedTB.structured_traceback(
  File "C:\Users\Pau